# Олимпиада НТИ - как выиграть за 30 строк

![](https://pp.userapi.com/c638719/v638719005/2f6da/mFHY83I9u2k.jpg)

### Машинное обучение и анализ данных - задача №2



Выделение из текста заранее известных сущностей - одна из самых распространенных

Середина двадцатого века один из самых сложных периодов нашей страны. По разным причинам (на войне, или в результате репрессий) пропало множество людей, о которых практически не осталось данных. Все данные которые мы можем выяснить это скупые карточки и журнальные записи, оставленные в разных формах. Множество общественных организаций занимается оцифровкой этих данных, чтобы наши соотечественники могли узнать судьбу своей семьи.   Однако, как писалось ранее, эти данные представляют собой скупые  и не структурированные заметки из разных журналов и форм, для того чтобы люди могли найти своих родственников необходимо из этих заметок выделить как можно больше классифицированной информации (сущностей)

Необходимо осуществить выделение сущностей из коротких текстов карточек. При этом не все записи содержат все сущности. Задача называется named entity recognition.
Список сущностей

Имя, фамилия, отчество, "никнейм или кличка", пол, год рождения, место рождения, работы, место проживания, был-ли арстован, день ареста, месяц ареста, год ареста,  кем арестован,  день суда, месяц суда, год суда, статья обвинения, наказание, был-ли растрелян, день расстрела, месяц расстрела, год расстрела, день реабилитации, месяц реабилитации, год реабилитации, источник, были ли дети, сам текст.


В качестве обучающей выборки представлены уже разобранные варианты.

# Решение

Задача состоит из трёх датасетов. Давайте их скачаем.

    Важно! Датасет из которого мы будем генерировать ответ нельзя использовать для вытаскивания фичей. На олимпиде выдавали кусок датасета на 5 минут, чтобы ты применил к нему свой код, а не руками доставал значения.

In [1]:
# Датасет с ответами
!wget https://tvorog.me/task2_answers.csv

--2017-04-06 15:21:52--  https://tvorog.me/task2_answers.csv
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся tvorog.me… 88.212.244.196
Подключение к tvorog.me|88.212.244.196|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 200800934 (191M) [application/octet-stream]
Сохранение в: «task2_answers.csv»

task2_answers.csv   100%[===================>] 191,50M  6,51MB/s    за 26s     

2017-04-06 15:22:19 (7,23 MB/s) - «task2_answers.csv» сохранён [200800934/200800934]



In [2]:
# Датасет на котором можно обучаться
!wget https://tvorog.me/task2_train.csv

--2017-04-06 15:23:42--  https://tvorog.me/task2_train.csv
Загружен сертификат CA «/etc/ssl/certs/ca-certificates.crt»
Распознаётся tvorog.me… 88.212.244.196
Подключение к tvorog.me|88.212.244.196|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 43217072 (41M) [application/octet-stream]
Сохранение в: «task2_train.csv»

task2_train.csv     100%[===================>]  41,21M  9,29MB/s    за 4,7s    

2017-04-06 15:23:47 (8,74 MB/s) - «task2_train.csv» сохранён [43217072/43217072]



In [ ]:
# Датасет на котором мы предсказываем ответы
!wget https://tvorog.me/